In [ ]:
import shutil
import tempfile

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
import swisstopopy
from rasterio import plot

In [ ]:
# spatial extent
agglom_extent_filepath = "../data/raw/agglom-extent.gpkg"

# buffer distance around the extent
buffer_dist = 500

# to download interim data and reuse it
working_dir = "land-working-dir"

# download parameters
alti3d_res = 2

# output files
dst_buildings_gdf_filepath = "../data/processed/buildings.gpkg"
dst_dem_filepath = "../data/processed/dem.tif"
dst_tree_canopy_filepath = "../data/processed/tree-canopy.tif"

In [ ]:
region = gpd.read_file(agglom_extent_filepath)["geometry"]
region = gpd.GeoSeries(region.buffer(buffer_dist), crs=region.crs)

In [ ]:
if working_dir is None:
    # create temporary directory to store the downloaded files (some of them will be
    # reused)
    working_dir = tempfile.mkdtemp()
pooch_retrieve_kwargs = {"path": working_dir}

In [ ]:
bldg_gdf = swisstopopy.get_bldg_gdf(region, **pooch_retrieve_kwargs)
bldg_gdf.to_file(dst_buildings_gdf_filepath)

ax = bldg_gdf.plot("height", legend=True)
cx.add_basemap(ax, crs=bldg_gdf.crs)

In [ ]:
swisstopopy.get_dem_raster(
    region,
    dst_dem_filepath,
    alti3d_res=alti3d_res,
    pooch_retrieve_kwargs=pooch_retrieve_kwargs,
)

fig, ax = plt.subplots()
with rio.open(dst_dem_filepath) as src:
    retted = plot.show(src, ax=ax)
fig.colorbar(retted.get_images()[0], ax=ax)

In [ ]:
swisstopopy.get_tree_canopy_raster(
    region, dst_tree_canopy_filepath, pooch_retrieve_kwargs=pooch_retrieve_kwargs
)

In [ ]:
if working_dir.startswith(tempfile.gettempdir()):
    # remove the temporary directory
    shutil.rmtree(working_dir)